In [1]:
from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note

import glob

def get_notes(glob_path):
    notes = []
    
    print(len(glob.glob(glob_path)))
    for input_file in glob.glob(glob_path):
        print(input_file)
        midi = converter.parse(input_file)

        parts = instrument.partitionByInstrument(midi)
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes




notes = get_notes('bowser/*.mid')
print(notes)
            
# get amount of pitch names
n_vocab = len(set(notes))

1
bowser/solfeggietto.mid
['E-3', 'C3', 'E-3', 'G3', 'C4', 'E-4', 'D4', 'C4', 'B3', 'G3', 'B3', 'D4', 'G4', 'F4', 'E-4', 'D4', 'E-4', 'C4', 'E-4', 'G4', 'C5', 'E-5', 'D5', 'C5', 'D5', 'C5', 'B4', 'A4', 'G4', 'F4', 'E-4', 'D4', 'E-4', 'C4', 'E-4', 'G4', 'C5', 'E-5', 'D5', 'C5', 'B4', 'G4', 'B4', 'D5', 'G5', 'F5', 'E-5', 'D5', 'E-5', 'C5', 'E-5', 'G5', 'C6', 'E-6', 'D6', 'C6', 'D6', 'C6', 'B5', 'A5', 'G5', 'F5', 'E-5', 'D5', 'E-5', 'C5', 'G4', 'E-4', 'C4', 'C6', 'G5', 'E-5', 'G#5', 'F3', 'G#3', 'C4', 'F4', 'G#4', 'C5', 'E-5', 'D5', 'B-4', 'F4', 'D4', 'B-3', 'B-5', 'F5', 'D5', 'G5', 'E-3', 'G3', 'B-3', 'D4', 'G4', 'B-4', 'D5', 'C5', 'A4', 'G#4', 'A4', 'C5', 'A4', 'G#4', 'A4', 'E-5', 'C5', 'G4', 'A4', 'E-5', 'C5', 'G4', 'A4', 'D5', 'C5', 'F#4', 'A4', 'A5', 'C5', 'F#4', 'A4', 'F#5', 'C5', 'D4', 'A4', 'C5', 'A4', 'F#4', 'C4', 'G4', 'G2', 'B-2', 'D3', 'G3', 'B-3', 'A3', 'G3', 'F#3', 'D3', 'F#3', 'A3', 'D4', 'C4', 'B-3', 'A3', 'B-3', 'G3', 'B-3', 'D4', 'G4', 'B-4', 'A4', 'G4', 'A4', 'G4', 'F#4

In [2]:
import numpy
from keras.utils import np_utils

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)



network_input, network_output = prepare_sequences(notes, n_vocab)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        128,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(128))
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model



model = create_network(network_input, n_vocab)

In [4]:
from keras.callbacks import ModelCheckpoint
import h5py

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=1, batch_size=6, callbacks=callbacks_list)


    
train(model, network_input, network_output)
print (network_output)

Epoch 1/1
431/431 [==============================] - 31s 71ms/step - loss: 3.8202
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
import pickle

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = numpy.append(pattern,index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

""" Generate a piano midi file """
#load the notes used to train the model
with open('data/notes', 'rb') as filepath:
    notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))


prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
print(prediction_output)

['1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8', '1.2.6.8'

In [6]:
from music21 import stream

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_outputy.mid')


create_midi(prediction_output)
print('donezo')

donezo
